
# **Часть 2**

Загрузим сохранённую модель с диска. Для этого подключим диск:


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install git+git://github.com/huggingface/transformers/

  Cloning git://github.com/huggingface/transformers/ to /tmp/pip-req-build-hj66su3d
  Running command git clone -q git://github.com/huggingface/transformers/ /tmp/pip-req-build-hj66su3d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 17.6MB/s 
     |████████████████████████████████| 2.9MB 46.5MB/s 
  Created wheel for transformers: filename=transformers-4.0.0rc1-cp36-none-any.whl size=1349466 sha256=1aa95bdd2450e9c843a494f7ebfa18e0c1c0473c04a9e6631b61fe064123f810
  Stored in directory: /tmp/pip-ephem-wheel-cache-jot8wh8h/wheels/dc/e5/1e/3a2977a646558fca07585cadcf56aa4a910e995ba945961c4e
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=cba105590305490864ec97aa5b479ed68087033275439c4ed9ae0b03e3efa61a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19

In [6]:
!git clone git://github.com/huggingface/transformers

fatal: destination path 'transformers' already exists and is not an empty directory.


In [7]:
import dataclasses
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Dict, Optional

import numpy as np
import torch

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, EvalPrediction
from transformers import (
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
)

logging.basicConfig(level=logging.INFO)

In [8]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )

In [9]:
model_args = ModelArguments(
    model_name_or_path="/content/drive/MyDrive/rudr-ADR-baseline-model",
)

Не забудем поставить штатный токенайзер.

In [11]:
config = AutoConfig.from_pretrained(
    model_args.model_name_or_path,
    num_labels=2,
    output_attentions=False  #can chage it to True later, when visualizing
)
tokenizer = AutoTokenizer.from_pretrained(
    "cimm-kzn/rudr-bert",
)
model = AutoModelForSequenceClassification.from_pretrained(
    model_args.model_name_or_path,
    config=config,
)

INFO:filelock:Lock 140132784969040 acquired on /root/.cache/torch/transformers/9954def1d4c7bb48365f9706ebda64b80ed7813dc6550a959bfb10241e4cd7a4.90bf1aefddf1893db3482a927e426e5dd3f5d5cf89b1ecc622660ed10e445bef.lock


INFO:filelock:Lock 140132784969040 released on /root/.cache/torch/transformers/9954def1d4c7bb48365f9706ebda64b80ed7813dc6550a959bfb10241e4cd7a4.90bf1aefddf1893db3482a927e426e5dd3f5d5cf89b1ecc622660ed10e445bef.lock
INFO:filelock:Lock 140132784968760 acquired on /root/.cache/torch/transformers/aaff3ba14d214d0da21b2418dce85afce2dfbebc0aae0ea3c4f5a9b88e6659ef.1b9d1283e3143d6e927a565444c4ebf167691824d119046b961ed5e9533ac93c.lock


INFO:filelock:Lock 140132784968760 released on /root/.cache/torch/transformers/aaff3ba14d214d0da21b2418dce85afce2dfbebc0aae0ea3c4f5a9b88e6659ef.1b9d1283e3143d6e927a565444c4ebf167691824d119046b961ed5e9533ac93c.lock


INFO:filelock:Lock 140132784968760 acquired on /root/.cache/torch/transformers/f7d0a08ecfd844d9d0066381bac30eb51bbb6a1dacf1c0ad03b87c98f72a6cb5.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d.lock


INFO:filelock:Lock 140132784968760 released on /root/.cache/torch/transformers/f7d0a08ecfd844d9d0066381bac30eb51bbb6a1dacf1c0ad03b87c98f72a6cb5.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d.lock
INFO:filelock:Lock 140132784968760 acquired on /root/.cache/torch/transformers/4acce3a63783fc5d0954177487821b7592b3b45001c26605384b9e9965979785.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b.lock


INFO:filelock:Lock 140132784968760 released on /root/.cache/torch/transformers/4acce3a63783fc5d0954177487821b7592b3b45001c26605384b9e9965979785.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b.lock


# Визуализация механизма Внимания

Чтобы объяснить предсказания, сделанные моделью, можем попытаться визуализировать внутренние веса механизма Внимания. Для этого воспользуемся библиотекой ```bertviz```.

In [12]:
!test -d bertviz_repo && echo "FYI: bertviz_repo directory already exists, to pull latest version uncomment this line: !rm -r bertviz_repo"
# !rm -r bertviz_repo # Uncomment if you need a clean pull from repo
!test -d bertviz_repo || git clone https://github.com/jessevig/bertviz bertviz_repo
if not 'bertviz_repo' in sys.path:
  sys.path += ['bertviz_repo']
!pip install regex

Cloning into 'bertviz_repo'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 1077 (delta 0), reused 2 (delta 0), pack-reused 1074
Receiving objects: 100% (1077/1077), 100.00 MiB | 25.57 MiB/s, done.
Resolving deltas: 100% (687/687), done.


In [13]:
def call_html():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/5.7.0/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
        '''))

In [14]:
sentence_a = "феназепам вызывает сонливость"
sentence_b = "тамифлю провоцирует усталость"
inputs = tokenizer.encode_plus(sentence_a,
                               sentence_b,
                               return_tensors='pt',
                               add_special_tokens=True,
                               padding=True)

In [15]:
cuda_check = inputs['input_ids'].is_cuda
cuda_check

False

In [17]:
model.to(device='cuda')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [18]:
token_type_ids = inputs['token_type_ids']
input_ids = inputs['input_ids']
model.config.output_attentions = True
attention = model(input_ids.to(device='cuda'),
                  token_type_ids=token_type_ids.to(device='cuda'))[-1]
input_id_list = input_ids[0].tolist() # Batch index 0
tokens = tokenizer.convert_ids_to_tokens(input_id_list)

Наибольший интерес представляют те графики, в которых связи соединяют два токена в разных предложениях, при этом токены обычные, не special tokens. Такие связи в основном представлены в первом ряду картинки: можно видеть, что они не столь сильные, как остальные. Это объяснимо: модель гораздо лучше умеет предсказывать по токену следующий токен (поэтому так много графиков с параллельными рёбрами), а также почти любой токен может следовать после специальных (старт и сепаратор, на картинке это графики с яркими лучами из углов/середины долей). Семантическая связь между подлежащими или сказуемыми в двух разных предложениях - точнее, связь между определёнными морфемами, ведь токенайзер делит слова на части - гораздо более тонкая, однако модель улавливает и её. По сути, именно построение таких связей потенциально способно улучшить классификацию текстов.

In [19]:
from bertviz import model_view

call_html()
model_view(attention, tokens)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Библиотека transformers перестала использовать словари для вывода Внимания. Насколько я понял из открытой issue (https://github.com/jessevig/bertviz/issues/31) именно это делает код ```neuron_view``` нерабочим.

In [ ]:
from bertviz.neuron_view import show
show(model.to(device="cpu"), 'bert', tokenizer, sentence_a, sentence_b)

TypeError: ignored